In [100]:
# Ref:
# YouTube Video
# Sam Witteveen
# Creating an AI Agent with LangGraph Llama 3 & Groq
# https://www.youtube.com/watch?v=lvQ96Ssesfk

In [61]:
!pip -q install groq
!pip -q install tavily-python

In [77]:
import json
import os
import re
from google.colab import userdata

# Define the API clients

In [64]:
from groq import Groq
from tavily import TavilyClient

groq_client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

tavily_client = TavilyClient(api_key=userdata.get('TAVILY_API_KEY'))

# What is the objective?

- Get an email from a customer
- Categorize the email
- Decide if research is needed before replying
- Do research if needed
- Write a draft reply

# Create a list of agents

AGENTS
1. email_cat_agent
2. router_agent (route to research_agent or email_writer_agent)
3. research_agent
4. email_writer_agent

TOOLS
- Tavily search (used by research_agent)


# Helper functions

In [65]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [66]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



In [113]:
def replace_items_in_string(input_string):

    """
    Sometimes models produce json style output strings that cannot be parsed.
    This functon extracts the "value" from the json string.
    Extracts the value from a json string.
    Works only if the json string has one key/value pair.
    """

    # The items in this list get replaced with nothing.
    # The key needs to be added to this list.
    items_to_replace = ["```", "json", "{", "}", '"email_draft": "',
                        '"translation": "', '"#"', '"category": "', '"router_decision": "', "\\r"]

    modified_string = input_string
    for item in items_to_replace:
        regex = re.compile(re.escape(item))  # Create a regex pattern for each item
        modified_string = regex.sub("", modified_string)  # Replace item with an empty string

    modified_string = modified_string.strip()

    # Slice to get the string from the start up to the second last character
    if len(modified_string) > 1:
        modified_string = modified_string[:-1]


    return modified_string



# Example usage
input_string = '{"email_draft": "some-value"}'
print(replace_items_in_string(input_string))

some-value


# Set up the LLM

In [67]:
def make_llm_api_call(message_history):

    """
    Makes a call to the Llama3 model on Groq.
    Args:
        message_history (List of dicts): The message history
    Returns:
        response_text: (str): The text response from the LLM
    """

    response = groq_client.chat.completions.create(
                        messages=message_history,
                        model="llama3-70b-8192",
                    )

    response_text = response.choices[0].message.content

    return response_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

response = make_llm_api_call(message_history)

print(response)

My name is Molly!


# Set up the tools



In [68]:
def run_tavily_search(query, num_results=5):

    """
    Uses the Tavily API to run a web search
    Args:
        query (str): The user query
        num_results (int): Num search results
    Returns:
        tav_response (json string): The search results in json format
    """

    # For basic search:
    tav_response = tavily_client.search(query=query, max_results=num_results)

    return tav_response



# Example

query = "How much does a bulldog weigh?"

results = run_tavily_search(query, num_results=2)

# Use this str output in the system message example below
# Use this instead of the Eisenhower example
print(results)

{'query': 'How much does a bulldog weigh?', 'follow_up_questions': None, 'answer': None, 'images': None, 'results': [{'title': 'English Bulldog Growth & Weight Chart: Everything You Need To ... - Pawlicy', 'url': 'https://www.pawlicy.com/blog/english-bulldog-growth-and-weight-chart/', 'content': 'According to Care.com, puppies reach about 75% of their adult height at six months old. This will be around 10-13 inches tall for a male English Bulldog and approximately 9-11 inches tall for a female English Bulldog. As for weight, a 6-month-old male English Bulldog will weigh about 33 to 37 pounds, while a 6-month-old female English Bulldog ...', 'score': 0.93752, 'raw_content': None}, {'title': 'English Bulldog Growth Chart: Male And Female Weight & Height', 'url': 'https://www.caninejournal.com/english-bulldog-growth-chart/', 'content': 'Male and female dogs have similar weights throughout puppyhood but differ slightly in adulthood. A full-grown male weighs around 51-55 pounds, while femal

# Set up the Agents

In [103]:
def run_email_cat_agent(email):

    print("---EMAIL CAT AGENT---")

    system_message = """
    You are a Email Categorizer Agent, You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way.

    You will be provided with an EMAIL. Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
    price_equiry - used when someone is asking for information about pricing \
    customer_complaint - used when someone is complaining about something \
    product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
    customer_feedback - used when someone is giving feedback about a product \
    off_topic when it doesnt relate to any other category \

    Output a single category only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic').
    Format your output as a JSON string with a key named "category".
    eg:
    {
    "category": "price_enquiry"
    }
    """

    # Create the "user" role content
    input = f"""
    EMAIL: {email}
    """

    # Create the message history that includes system and user roles
    message_history = create_message_history(system_message, input)

    # Prompt the llm
    response = make_llm_api_call(message_history)

    # Extract the response from the json string
    response = replace_items_in_string(response)

    print("Email category:", response)

    return response


# Example

email = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

email_cat = run_email_cat_agent(email)

#print(email_cat)

---EMAIL CAT AGENT---
Email category: customer_feedback


In [119]:
def run_router_agent(email, email_cat):

    print("---ROUTER AGENT---")

    system_message = """
    You are an expert at reading the initial email and routing web search or directly to a draft email. \n
    You will be provided with a CUSTOMER_EMAIL and an EMAIL_CATEGORY.

    Use the following criteria to decide how to route the email: \n\n

    If the initial email only requires a simple response choose 'to_email_writer_agent'.
    If the email is just saying thank you etc then choose 'to_email_writer_agent'.
    Otherwise, use to_research_agent.

    Give a binary choice 'to_research_agent' or 'to_email_writer_agent'. Return the a JSON with a single key 'router_decision' and
    no premable or explaination. Use both the initial email and the email category to make your decision.
    """

    input = f"""
    CUSTOMER_EMAIL: {email}
    EMAIL_CATEGORY: {email_cat}
    """

    # Create the message history that includes system and user roles
    message_history = create_message_history(system_message, input)

    # Prompt the llm
    response = make_llm_api_call(message_history)

    # Extract the response from the json string
    response = replace_items_in_string(response)

    print("Router decision:", response)

    return response



# Example

email = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

email_cat = run_email_cat_agent(email)

router_decision = run_router_agent(email, email_cat)

#router_decision

---EMAIL CAT AGENT---
Email category: customer_feedback
---ROUTER AGENT---
Router decision: to_email_writer_agent


In [109]:
def run_research_agent(email, email_cat):

    print("---RESEARCH AGENT---")

    system_message = """
    You are a master at working out the best keywords to search for in a web search to get the best info for the customer.

    given the CUSTOMER_EMAIL and EMAIL_CATEGORY. Work out the best keywords that will find the best
    info for helping to write a response to the customer email.

    Return a JSON with a single key 'keywords' with no more than 5 keywords and no premable or explaination.
    """

    input = f"""
    CUSTOMER_EMAIL: {email}
    EMAIL_CATEGORY: {email_cat}
    """

    message_history = create_message_history(system_message, input)

    response = make_llm_api_call(message_history)

    response = json.loads(response)

    keywords_list = (response['keywords'])

    # Run searches using the keywords

    print("Search keywords:\n", keywords_list)
    print("Running tavily search...")

    research_info_list = []

    for query in keywords_list:

        results = run_tavily_search(query, num_results=2)
        research_info_list.append(results)

    print("Research complete.")

    return research_info_list




# Example

email = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

email_cat = run_email_cat_agent(email)

research_list = run_research_agent(email, email_cat)

#research_list[0]

---EMAIL CAT AGENT---
Email category: customer_complaint
---RESEARCH AGENT---
Search keywords:
 ['Arizona weather April', 'cloudy resort weather', 'average weather conditions', 'weather complaints', 'vacation weather expectations']
Running tavily search...
Research complete.


In [114]:
def run_email_writer_agent(email, email_cat, research_info_list):

    print("---EMAIL WRITER AGENT---")

    system_message = """
    You are the Email Writer Agent take the CUSTOMER_EMAIL below  from a human that has emailed our company email address, the email_category \
    that the categorizer agent gave it and the research from the research agent and \
    write a helpful email in a thoughtful and friendly way.

    If the customer email is 'off_topic' then ask them questions to get more information.
    If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
    If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
    If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
    If the customer email is 'price_equiry' then try to give the pricing info they requested.

    You never make up information that hasn't been provided by the research_info or in the initial_email.
    Always sign off the emails in appropriate manner and from Sarah the Resident Manager.

    Return a JSON string with a single key 'email_draft' and no premable or explaination.
    """

    input = f"""
    CUSTOMER_EMAIL: {email}
    EMAIL_CATEGORY: {email_cat}
    RESEARCH_INFO: {research_info_list}
    """

    # Create the message history that includes system and user roles
    message_history = create_message_history(system_message, input)

    # Prompt the llm
    response = make_llm_api_call(message_history)

    # Extract the response from the json string
    response = replace_items_in_string(response)

    # Remove leading and trailing spaces
    response = response.strip()

    print("Email writing complete.")
    print("Final email:\n\n", response)

    return response



# Example


email = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

email_cat = run_email_cat_agent(email)

research_list = run_research_agent(email, email_cat)

final_email = run_email_writer_agent(email, email_cat, research_list)

#print(final_email)


---EMAIL CAT AGENT---
Email category: customer_complaint
---RESEARCH AGENT---
Search keywords:
 ['Arizona weather April', 'resort weather forecast', 'cloudy weather complaints', 'weather disappointment', 'travel weather expectations']
Running tavily search...
Research complete.
---EMAIL WRITER AGENT---
Email writing complete.
Final email:

 Dear George,

Thank you for reaching out to us about your recent experience at our resort. I apologize that the weather didn't meet your expectations. I understand that cloudy weather can be disappointing, especially when you're hoping for sunshine.

According to our research, in April, Arizona typically experiences mild temperatures with average highs in the mid-70s to mid-80s Fahrenheit. However, it's not uncommon to have some cloudy days during this time of year.

I appreciate your feedback and would like to assure you that we value your opinions and concerns. We will take your comments into consideration to improve our services for future guests

# Run the system

In [122]:
email1 = """HI there, \n
I am emailing to say that the resort weather was way to cloudy and overcast. \n
I wanted to write a song called 'Here comes the sun but it never came'

What should be the weather in Arizona in April?

I really hope you fix this next time.

Thanks,
George
"""

email2 = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""


# Initialize variables
email = email1
research_list = ""

email_cat = run_email_cat_agent(email)

router_decision = run_router_agent(email, email_cat)

if router_decision == "to_research_agent":
    research_list = run_research_agent(email, email_cat)

final_email = run_email_writer_agent(email, email_cat, research_list)

---EMAIL CAT AGENT---
Email category: customer_complaint
---ROUTER AGENT---
Router decision: to_research_agent
---RESEARCH AGENT---
Search keywords:
 ['Arizona weather April', 'overcast weather', 'resort weather', 'Arizona climate', 'April weather forecast']
Running tavily search...
Research complete.
---EMAIL WRITER AGENT---
Email writing complete.
Final email:

 Dear George,

Thank you for reaching out to us about your recent experience at our resort. I'm so sorry to hear that the weather didn't quite live up to your expectations. We understand how frustrating it can be when the sun doesn't shine as brightly as we'd like.

Regarding your question about the weather in Arizona in April, our research suggests that it's usually mild and pleasant, with average high temperatures ranging from the mid-70s to the mid-80s Fahrenheit (23-30°C). Of course, weather can be unpredictable, and it's not uncommon to experience some cloudy days during that time of year.

We appreciate your feedback and